# Data segmentation and visualization
This notebook segments and visualizes data from the CITRIFIED experiments.

The data is taken from the `preprocessed_transformed_data` directory of the CITRIFIED repository (not included in source control). The `preprocessed_transformed_data`
directory should have the following structure:
```bash
data
└──preprocessed_transformed_data
  └──experiment
     └──fruit
        └──cut_quality
           ├──fruit_cutquality_run1.csv
           ├──fruit_cutquality_run2.csv
           ├──...
           └──...
```
Where `experiment`, `fruit`, `cut_quality`, and `runX` can have arbitrary names (for example `december`, `orange`, `good`, and `1`).
You have to configure these names below, under **Data choice**.

The `csv` files contain preprocessed and transformed data from the previous step (see `preprocess_transform_data.ipynp` notebook).
In detail, they contain Optitrack and FT sensor data of desired frames and wrench components sampled at the same time points.

Until now, the segmentation happens based on the gradient of the cutting force. Only the segment between the max and min value of the gradient is kept.

## Data choice

In [ ]:
experiment = 'december'

In [ ]:
fruit = 'orange'

In [ ]:
cut_qualities = ['good'] # good / shallow / deep

In [ ]:
frames = ['ExactoKnife']

In [ ]:
force_components = ['force'] # force and/or torque

### Imports


In [ ]:
import itertools
import os
from os.path import join
import numpy as np
import pandas as pd

from scipy.spatial.transform import Rotation

from plotly.subplots import make_subplots
import plotly.graph_objects as go


### Plot functions

In [ ]:
def plot_data(x, y, header=""):
    '''Plot data from x and y with subplots.'''
    fig = make_subplots(rows=y.shape[1], cols=1,x_title='Time',)

    for index in range(y.shape[1]):
        fig.append_trace(go.Scatter(
            x=x,
            y=y.iloc[:,index],
            name=y.columns[index],
        ), row=index+1, col=1)

    fig.update_layout(height=600, width=600, title_text=header)
    fig.show()

### Export

In [ ]:
def export_run(run, data, folder_name):
    '''Export data from data frame 'data' to csv file.'''
    export_folder = join('../..', 'data', folder_name)

    if not os.path.isdir(export_folder):
        os.makedirs(export_folder)

    filename = os.path.split(run)
    data.to_csv(join(export_folder, filename[1]), index=False)


## Data extraction

In [ ]:
data_folder = join('../..', 'data', 'preprocessed_transformed_data', experiment)
print(data_folder)

In [ ]:
folders  = {cq: join(data_folder, fruit, cq) for cq in cut_qualities}
print(folders)

In [ ]:
all_runs = {cq: [join(folder, file) for file in os.listdir(folder)] for cq, folder in folders.items()}

In [ ]:
opt_position_header = list(itertools.chain.from_iterable((f + '_x', f + '_y', f + '_z') for f in frames))

In [ ]:
ft_desired_header = list(itertools.chain.from_iterable((v + '_x', v + '_y', v + '_z') for v in force_components))

## Segmentation

In [ ]:
timeseries = {}
for cq, runs in all_runs.items():
    timeseries[cq] = []
    for r in runs:
        print('Processing run ' + r)
        data = pd.read_csv(r)
        
        # cut trajectory based on gradient of force along cutting direction
        gradient = [(b - a) for a, b in
                        zip(data['force_x'][:-1], data['force_x'][1:])]
        max_index = gradient.index(max(gradient))
        min_index = gradient.index(min(gradient))
        data_segmented = data.loc[max_index:min_index]
        data_segmented.reset_index(inplace=True)
        
        #plot_data(data_segmented['relative_time'], data_segmented[ft_desired_header])

        # set start point of cut to (0,0,0) and calculate displacement along cut
        for column in opt_position_header:
            data_segmented.loc[:, column] -= data_segmented.loc[0, column]
            
        displacement = [np.linalg.norm(x) for x in
                            zip(data_segmented["ExactoKnife_x"], data_segmented["ExactoKnife_y"],
                                data_segmented["ExactoKnife_z"])]
        data_segmented["displacement"] = displacement
        
        export_run(r, data_segmented, join('segmented_data', experiment, fruit, cq))
        timeseries[cq].append(data_segmented)       
        

## Visualization options

In [ ]:
desired_nominal_time = 8

## Data visualization

In [ ]:
position_time_fig = make_subplots(rows=3, cols=1,x_title='time [s]')
force_time_fig = make_subplots(rows=3, cols=1,x_title='time [s]')
force_displacement_fig = make_subplots(rows=3, cols=1,x_title='relative displacement from cut start [m]')

for cq, runs in timeseries.items():
    for r in runs:
        # stretch / shorten cut to desired nominal time
        total_time = r['relative_time'].iloc[-1] - r.loc[0, 'relative_time']
        r.loc[:, 'relative_time'] = (r['relative_time'] - r.loc[0, 'relative_time']) / total_time * desired_nominal_time

        angle = np.arctan(r['ExactoKnife_x'].iloc[-20] / r['ExactoKnife_y'].iloc[-20])
        R = Rotation.from_euler("z", angle).as_matrix()
        r.loc[:, 'ExactoKnife_x'] = [(R.dot(x))[0] for x in
                                      zip(r['ExactoKnife_x'],
                                          r['ExactoKnife_y'],
                                          r['ExactoKnife_z'])]
        r.loc[:, 'ExactoKnife_y'] = [(R.dot(x))[1] for x in
                                      zip(r['ExactoKnife_x'],
                                          r['ExactoKnife_y'],
                                          r['ExactoKnife_z'])]
    
        for i, pos in enumerate(opt_position_header):
            position_time_fig.append_trace(go.Scatter(
                    x=r['relative_time'],
                    y=r[pos],
                    ), row=i+1, col=1)
            
        for i, force in enumerate(ft_desired_header):
            force_time_fig.append_trace(go.Scatter(
                    x=r['relative_time'],
                    y=r[force],
                    ), row=i+1, col=1)

        for i, force in enumerate(ft_desired_header):
            force_displacement_fig.append_trace(go.Scatter(
                    x=r['displacement'],
                    y=r[force],
                    mode='markers',
                    showlegend=False,
                    ), row=i+1, col=1)
        

    position_time_fig.update_yaxes(title_text='position_x [m]', row=1, col=1)
    position_time_fig.update_yaxes(title_text='position_y [m]', row=2, col=1)
    position_time_fig.update_yaxes(title_text='position_z [m]', row=3, col=1)
    position_time_fig.update_layout(height=600, width=800, title_text='-'.join([fruit, cq]),showlegend=False)
    position_time_fig.show()
    
    force_time_fig.update_yaxes(title_text='force_x [N]', row=1, col=1)
    force_time_fig.update_yaxes(title_text='force_y [N]', row=2, col=1)
    force_time_fig.update_yaxes(title_text='force_z [N]', row=3, col=1)
    force_time_fig.update_layout(height=600, width=800, title_text='-'.join([fruit, cq]),showlegend=False)
    force_time_fig.show()
    
    force_displacement_fig.update_yaxes(title_text="force_x [N]", row=1, col=1)
    force_displacement_fig.update_yaxes(title_text="force_y [N]", row=2, col=1)
    force_displacement_fig.update_yaxes(title_text="force_z [N]", row=3, col=1)
    force_displacement_fig.update_layout(height=800, width=1000, title_text='-'.join([fruit, cq]))
    force_displacement_fig.show()
    